In [ ]:
#create dataframe

In [1]:
df_hotel = spark.createDataFrame(
    [
        (5001, 21),
        (5001, 22),
        (5002,21)
    ],
    ["hotel_id", "advertiser"]
)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1643446030525_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df_hotel.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+
|hotel_id|advertiser|
+--------+----------+
|    5001|        21|
|    5001|        22|
|    5002|        21|
+--------+----------+

In [3]:
df_hotel.createOrReplaceTempView("hotel")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sqlDF = spark.sql("SELECT * FROM hotel")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+
|hotel_id|advertiser|
+--------+----------+
|    5001|        21|
|    5001|        22|
|    5002|        21|
+--------+----------+

In [5]:
df_clicks =spark.createDataFrame(
    [
        ("US",21,5001,100),
        ("US",22,5001,50),
        ("US",21,5002,10),
        ("IT",22,5001,5)
    ],
    ["country" ,"advertiser","hotel_id","clicks" ]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_clicks.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+--------+------+
|country|advertiser|hotel_id|clicks|
+-------+----------+--------+------+
|     US|        21|    5001|   100|
|     US|        22|    5001|    50|
|     US|        21|    5002|    10|
|     IT|        22|    5001|     5|
+-------+----------+--------+------+

In [7]:
df_clicks.createOrReplaceTempView("clicks")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#To have a better visualization of the case, I added more data in this data frame starting from the 3rd row.
df_bookings =spark.createDataFrame(
    [
        ("US",21,5001,10,1000),
        ("DE",21,5002,1,200),
        ("US",23,5003,2,500),
        ("US",23,5003,2,1500),
        ("US",26,5005,6,600),
        ("DE",25,5003,2,500),
        ("IT",3,5009,50,700),
        ("IT",25,5003,30,1000)
    ],
    ["country" ,"advertiser","hotel_id","bookings","booking_volume"]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_bookings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+--------+--------+--------------+
|country|advertiser|hotel_id|bookings|booking_volume|
+-------+----------+--------+--------+--------------+
|     US|        21|    5001|      10|          1000|
|     DE|        21|    5002|       1|           200|
|     US|        23|    5003|       2|           500|
|     US|        23|    5003|       2|          1500|
|     US|        26|    5005|       6|           600|
|     DE|        25|    5003|       2|           500|
|     IT|         3|    5009|      50|           700|
|     IT|        25|    5003|      30|          1000|
+-------+----------+--------+--------+--------------+

In [10]:
df_bookings.createOrReplaceTempView("bookings")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

###  1.Total number of clicks per hotel

In [11]:
sqlDF = spark.sql("\
        SELECT hotel_id,sum(clicks) AS total_clicks FROM clicks\
        GROUP BY hotel_id\
        ORDER BY hotel_id\
")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|hotel_id|total_clicks|
+--------+------------+
|    5001|         155|
|    5002|          10|
+--------+------------+

###  2.Top advertiser per country in terms of booking volume

In [47]:
sqlDF = spark.sql("\
            SELECT advertiser,country,SUM(booking_volume) AS total_volume\
            FROM bookings\
            GROUP BY country,advertiser\
            ORDER BY country\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("total_volume_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------------+
|advertiser|country|total_volume|
+----------+-------+------------+
|        21|     DE|         200|
|        25|     DE|         500|
|         3|     IT|         700|
|        25|     IT|        1000|
|        23|     US|        2000|
|        26|     US|         600|
|        21|     US|        1000|
+----------+-------+------------+

In [46]:
sqlDF = spark.sql("\
    SELECT advertiser,country,MAX(total_volume) over (PARTITION BY country) as max_volume\
    FROM top_volume_table\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("max_volume_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+----------+
|advertiser|country|max_volume|
+----------+-------+----------+
|        21|     DE|       500|
|        25|     DE|       500|
|         3|     IT|      1000|
|        25|     IT|      1000|
|        21|     US|      2000|
|        23|     US|      2000|
|        26|     US|      2000|
+----------+-------+----------+

In [54]:
sqlDF = spark.sql("\
SELECT max_volume_table.advertiser,max_volume_table.country,max_volume\
                  FROM max_volume_table\
                  JOIN total_volume_table\
                  ON max_volume_table.country = total_volume_table.country\
                  AND max_volume_table.advertiser=total_volume_table.advertiser\
                  AND max_volume_table.max_volume=total_volume_table.total_volume\
")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+----------+
|advertiser|country|max_volume|
+----------+-------+----------+
|        25|     DE|       500|
|        25|     IT|      1000|
|        23|     US|      2000|
+----------+-------+----------+

### 3.2nd advertiser per country in terms of booking volume

In [71]:
sqlDF = spark.sql(" \
SELECT * FROM(\
    SELECT advertiser,country,total_volume, rank() over (partition by country order by total_volume DESC) AS rank\
    FROM total_volume_table)\
WHERE rank=2\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("without_top_volume_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------------+----+
|advertiser|country|total_volume|rank|
+----------+-------+------------+----+
|        21|     DE|         200|   2|
|         3|     IT|         700|   2|
|        21|     US|        1000|   2|
+----------+-------+------------+----+

In [74]:
sqlDF = spark.sql("\
SELECT without_top_volume_table.advertiser,without_top_volume_table.country,without_top_volume_table.total_volume\
                  FROM without_top_volume_table\
                  JOIN total_volume_table\
                  ON without_top_volume_table.country = total_volume_table.country\
                  AND without_top_volume_table.advertiser=total_volume_table.advertiser\
                  AND without_top_volume_table.total_volume=total_volume_table.total_volume\
")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------------+
|advertiser|country|total_volume|
+----------+-------+------------+
|        21|     DE|         200|
|         3|     IT|         700|
|        21|     US|        1000|
+----------+-------+------------+

### 4.Booking conversion rate (bookings / clicks) for all hotels in trivago.advertiser_hotels

In [77]:
sqlDF = spark.sql("\
    SELECT hotel_id, sum(clicks) AS total_clicks FROM clicks\
    GROUP BY hotel_id\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("total_clicks_per_hotel")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|hotel_id|total_clicks|
+--------+------------+
|    5002|          10|
|    5001|         155|
+--------+------------+

In [78]:
sqlDF = spark.sql("\
    SELECT hotel_id, sum(bookings) AS total_bookings FROM bookings\
    GROUP BY hotel_id\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("total_bookings_per_hotel")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+
|hotel_id|total_bookings|
+--------+--------------+
|    5009|            50|
|    5002|             1|
|    5003|            36|
|    5005|             6|
|    5001|            10|
+--------+--------------+

In [79]:
sqlDF = spark.sql("\
    SELECT total_bookings_per_hotel.hotel_id\
    FROM total_bookings_per_hotel\
    UNION\
    SELECT total_clicks_per_hotel.hotel_id\
    FROM total_clicks_per_hotel\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("full_hotel_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|hotel_id|
+--------+
|    5009|
|    5002|
|    5003|
|    5005|
|    5001|
+--------+

In [80]:
sqlDF = spark.sql("\
    SELECT full_hotel_id.hotel_id,total_bookings,total_clicks,total_bookings/NULLIF(total_clicks,0) AS booking_conversion_rate\
    FROM full_hotel_id\
    full JOIN total_bookings_per_hotel ON total_bookings_per_hotel.hotel_id = full_hotel_id.hotel_id\
    full JOIN total_clicks_per_hotel ON total_clicks_per_hotel.hotel_id = full_hotel_id.hotel_id\
")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+------------+-----------------------+
|hotel_id|total_bookings|total_clicks|booking_conversion_rate|
+--------+--------------+------------+-----------------------+
|    5009|            50|        null|                   null|
|    5002|             1|          10|                    0.1|
|    5003|            36|        null|                   null|
|    5005|             6|        null|                   null|
|    5001|            10|         155|    0.06451612903225806|
+--------+--------------+------------+-----------------------+